In [1]:
import alpaca_trade_api as alpaca
from dotenv import load_dotenv
import os

In [ ]:
# initialize API from API keys in .env
load_dotenv()
api_key = os.environ['APCA-API-KEY-ID']
api_private_key = os.environ['APCA-API-SECRET-KEY']
api_base_url = 'https://paper-api.alpaca.markets'
api = alpaca.REST(api_key, api_private_key, api_base_url)

In [ ]:
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA3DYO3UIUR9',
    'accrued_fees': '0',
    'admin_configurations': {},
    'balance_asof': '2024-11-15',
    'bod_dtbp': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': '2024-11-18T15:46:34.388288Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 0,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'effective_buying_power': '200000',
    'equity': '100000',
    'id': 'ce3cfc2a-396f-46f1-b2c3-bd841a8e2582',
    'initial_margin': '0',
    'intraday_adjustments': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 2,
    'options_buying_power': '100000',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_reg_taf_fees': '0',
    'portfolio_value': '100000',
    